# Libraries

In [1]:
import scipy as sp
import numpy as np
import pandas as pd
import sklearn as sk
from collections import defaultdict

try:
    import py_entitymatching as em
except:
    !pip install py_entitymatching

try:
    import deepmatcher as dm
except:
    !pip uninstall preprocessing -y
    !pip uninstall fastai -y 
    !pip uninstall allennlp -y
    !pip install deepmatcher

if pd.__version__ != "1.2.4":
 !pip install pandas==1.2.4
 import pandas as pd
 print(f"Verify pandas=={pd.__version__}")
else:
 print(f"Verify pandas=={pd.__version__}")

import sys
import tensorflow as tf
from tensorflow.python.client import device_lib
import os
from imblearn.under_sampling import RandomUnderSampler
import time

Verify pandas==1.2.4


/usr/local/lib/python3.7/dist-packages/sklearn/externals/six.py:31: FutureWarning: The module is deprecated in version 0.21 and will be removed in version 0.23 since we've dropped support for Python 2.7. Please rely on the official version of six (https://pypi.org/project/six/).
  "(https://pypi.org/project/six/).", FutureWarning)
/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:144: FutureWarning: The sklearn.neighbors.base module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from sklearn.neighbors. Anything that cannot be imported from sklearn.neighbors is now part of the private API.
  warnings.warn(message, FutureWarning)


In [2]:
! git clone https://github.com/ljhoh1/Dissertation_Data_2

Cloning into 'Dissertation_Data_2'...
remote: Enumerating objects: 92, done.
remote: Counting objects: 100% (92/92), done.
remote: Compressing objects: 100% (64/64), done.
remote: Total 92 (delta 38), reused 80 (delta 28), pack-reused 0
Unpacking objects: 100% (92/92), done.
Checking out files: 100% (36/36), done.


# Data

In [3]:
path_Match_Cora = os.path.join('.', 'Dissertation_Data_2', 'cora_duplicates_data.tsv')
labelled_Cora = em.read_csv_metadata(path_Match_Cora)

path_Cora = os.path.join('.', 'Dissertation_Data_2', 'cora_data.tsv')

In [5]:
# Cora data
csv_table=pd.read_table(path_Cora,sep='\t')
csv_table.to_csv(os.path.join('.', 'Dissertation_Data_2', 'cora_data.csv'),index=False)
Cora = em.read_csv_metadata(os.path.join('.', 'Dissertation_Data_2', 'cora_data.csv'), key='id')

In [6]:
Cora.isna().sum()

address        1139
authors           3
booktitle       974
date           1374
editor         1422
id                0
institution    1631
journal        1244
month          1849
note           1807
pages           643
publisher      1173
tech           1729
title            41
type           1766
volume         1044
year            654
dtype: int64

In [7]:
for col in Cora.columns:
  Cora[col] = Cora[col].astype(str).str.replace('[^A-Za-z0-9 ]+', '')
  #Cora = Cora.str.replace('[^A-Za-z0-9 ]+', '')
  Cora[col] = Cora[col].astype(str).str.lower()
  #Cora.loc[Cora[col] == 'nan', [col]] = np.nan
Cora.head()
Cora['date'] = Cora['date'].astype(str).str.replace('[^0-9]+', '')
#Cora['yea'] = Cora['year']
Cora.year.fillna(Cora.date, inplace=True)
Cora.loc[Cora['year'] == '', ['year']] = np.nan
#del Cora['date']
del Cora['date']

Cora.rename(columns = {'type':'new_type'}, inplace = True)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: FutureWarning: The default value of regex will change from True to False in a future version.
  
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:7: FutureWarning: The default value of regex will change from True to False in a future version.
  import sys


# Blocking

In [8]:
# Blocking - Cora/Cora
# Get tokenizers and similarity function
block_t = em.get_tokenizers_for_blocking()
block_s = em.get_sim_funs_for_blocking()

# Get attributes
atypes1 = em.get_attr_types(Cora)
atypes2 = em.get_attr_types(Cora)

# Get correspondence
block_c = em.get_attr_corres(Cora, Cora)

#Get Features
block_f = em.get_features(Cora, Cora, atypes1, atypes2, block_c, block_t, block_s)

cols_cora = [i for i in Cora.columns]

In [9]:
# Creating a rule based blocker
rb = em.RuleBasedBlocker()

cols_cora = [i for i in Cora.columns]
rule = ['title_title_jac_qgm_3_qgm_3(ltuple, rtuple) < 0.35']
rb.add_rule(rule, feature_table=block_f)
K4 = rb.block_tables(Cora, Cora,
                   l_output_attrs=cols_cora, 
                   r_output_attrs=cols_cora,
                   n_jobs=1)

# Creating a rule based blocker
#rb = em.RuleBasedBlocker()

#rule = ['authors_authors_jac_qgm_3_qgm_3(ltuple, rtuple) < 0.1']
#rb.add_rule(rule, feature_table=block_f)
#K4 = rb.block_candset(K3, n_jobs=1)

0% [##############################] 100% | ETA: 00:00:00
Total time elapsed: 00:00:02


Finding pairs with missing value...


# Helper Functions

In [10]:
def labelling(train, labelled, key = "idDBLP", val = "idScholar"):
  '''
  Create dictionary for checking how many matches are retained and perform labelling
  Input:
  - train (pd.DataFrame): DataFrame object of blocked data used for training
  - lbelled (pd.DataFrame): The labelled DataFrame for labelling the train data
  - key (string): which table id in blocked "train" to use as key for labelling
  - val (string): which table id in blocked "train" to use as value for labelling
  Output:
  - train (pd.DataFrame): The labelled training dataframe
  '''
  train["id_comb"] = train["rtable_id"].astype(str) + train["ltable_id"].astype(str)
  labelled["id_comb"] = labelled[key].astype(str) + labelled[val].astype(str)

  K = train.merge(labelled, on='id_comb', how='left', indicator=True)
  K.loc[(K['_merge'] == "both"), 'label'] = 1
  K.loc[(K['_merge'] == "left_only") | (K['_merge'] == "right_only"), 'label'] = 0
  del K['_merge']
  del K['id1']
  del K['id2']
  del K['id_comb']
  return K

In [11]:
def scores(time, predictions, iteration, train_n, model, epochs=15):
  '''
  Report the most important metrics of performance from the predictions made by the model
  Input:
  - time (float): time the model ran for
  - predictions (pd.DataFrame): contains the output features and column "match_score", containing the prediction information
  - iteration (int64): the current iteration if applicable
  - train_n (int64): the number of samples used in training
  - model (string): the model that was used
  - epochs (int64): the number of epochs the model was trained for
  Output:
  - dict (dictionary): dictionary containing the performance information  
  '''
  # casting prediction scores as binary labels
  predictions['match_prediction'] = predictions['match_score'].apply(lambda score: 1 if score >= 0.5 else 0)
  # Reset index as Magellan requires the key to be a column in the table
  predictions.reset_index(inplace=True)
  # Update metadata in the catalog. This information can later be used by triggers to modify the labels from 
  # the learning-based matcher 
  em.set_key(predictions, '_id')
  em.set_fk_ltable(predictions, 'ltable_id')
  em.set_fk_rtable(predictions, 'rtable_id')
  # AUC-ROC metrics
  auc = sk.metrics.roc_auc_score(predictions['label'], predictions['match_prediction'], average = None)
  # Precision Metrics
  prec = em.eval_matches(predictions, 'label', 'match_prediction')
  # Creating dictionary of performance metrics to append to a DataFrame
  dict = {'Model':model, 'Time':time, 'Epochs':epochs,'Iteration':iteration, '#Training_samples':train_n,
                          'F1':prec['f1'], 'Precision':prec['precision'], 
                          'Recall':prec['recall'], 'AUC':auc}
  return dict
# Create a placeholder matrix to append precision scores to
prec_matrix = pd.DataFrame(columns=['Model', 'Time', 'Epochs', 'Iteration', '#Training_samples', 'F1', 'Precision', 'Recall', 'AUC'])

In [12]:
def subsampling(cand):
  '''
  Function to subsample to avoid unbalanced data
  Input:
  - cand (pd.DataFrame): contains the blocked (and transformed) data with "label" feature
  Output:
  - cand_resampled (pd.DataFrame): contains the resampled data to avoid imbalance
  '''
  # subsampling the candidate set for better balance
  # creating the explanatory and dependent feature data
  binary_X = cand.loc[:, cand.columns != "label"]
  binary_y = cand.loc[:, cand.columns == "label"]
  # initialising the RandomUnderSampler
  rus = RandomUnderSampler(random_state=0)
  # resampling to 0.5/0.5
  X_res, y_res = rus.fit_resample(binary_X, binary_y)
  res_labelled = pd.DataFrame(X_res)
  res_labelled.columns = cand.columns[0:-1]
  # take the index of resampled observations to subsample from the original
  sel_idx = list(res_labelled._id)
  cand_resampled = cand[cand["_id"].isin(sel_idx)]
  return cand_resampled

# Processing

In [13]:
# The path to the labeled data file.
csv_table=pd.read_table(path_Match_Cora, sep='\t')
csv_table.to_csv(os.path.join('.', 'Dissertation_Data_2', 'cora_labelled.csv'),index=False)
cora_labelled = em.read_csv_metadata(os.path.join('.', 'Dissertation_Data_2', 'cora_labelled.csv'))

In [14]:
cols = list(cora_labelled.columns)
a, b = cols.index('id1'), cols.index('id2')
cols[b], cols[a] = cols[a], cols[b]
cor_lab_rev = cora_labelled[cols]
cor_lab_rev.columns = ['id1', 'id2']
cora_labelled = cora_labelled.append(cor_lab_rev)

In [15]:
# Labelling the blocked data set
cand = labelling(K4, cora_labelled, key = "id1", val = "id2")
print("Proportion of matches retained after blocking:", round(sum(cand['label']) / cora_labelled.shape[0], 2))

# subsampling the candidate set for better balance
for col in cand.columns:
    if col != "_id" and col != "label":
      cand[col] = cand[col].astype(str).str.replace('[^A-Za-z0-9 ]+', '')
    else:
      cand[col] = cand[col].astype(int)

#cand = cand.sample(frac=0.5, random_state=1)

Proportion of matches retained after blocking: 1.0


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:8: FutureWarning: The default value of regex will change from True to False in a future version.
  


In [16]:
cora_labelled.shape

(129156, 3)

In [17]:
sum(cand['label'])

129102

In [18]:
# The directory where the data splits will be saved.
split_path = os.path.join('.', 'Dissertation_Data_2')
# Split labeled data into train, valid, and test csv files to disk, with the split ratio of 3:1:1.
dm.data.split(cand, split_path, 'train.csv', 'valid.csv', 'test.csv',
              [3, 1, 1],
              stratified=True)

In [19]:
#train_tosub = em.read_csv_metadata(os.path.join('.', 'Dissertation_Data_2', 'train.csv'))

#cand_resampled = train_tosub.sample(frac=0.6, random_state=1)
#cand_resampled.to_csv(os.path.join('.', 'Dissertation_Data_2', 'train.csv'),index=False)

In [20]:
print("Proportion of matches retained after blocking:", round(sum(cand['label']) / cand.shape[0], 2))

Proportion of matches retained after blocking: 0.62


In [21]:
# Load the training data files from the disk. Ignore the "left_id" and "right_id" 
# columns for data preprocessing.
# The 'use_magellan_convention' parameter asks deepmatcher to use Magellan's 
# naming convention for the left and right table column name prefixes 
# ("ltable_", and "rtable_"), and also to consider "_id" as the ID column.
train, validation, test = dm.data.process(
    path=os.path.join('.', 'Dissertation_Data_2'),
    cache='train_cache.pth',
    train='train.csv',
    validation='valid.csv',
    test='test.csv',
    use_magellan_convention=True,
    ignore_columns=('ltable_id', 'rtable_id'))


Reading and processing data from "./Dissertation_Data_2/train.csv"
0% [##############################] 100% | ETA: 00:00:00
Reading and processing data from "./Dissertation_Data_2/valid.csv"
0% [##############################] 100% | ETA: 00:00:00
Reading and processing data from "./Dissertation_Data_2/test.csv"
0% [##############################] 100% | ETA: 00:00:00INFO:deepmatcher.data.field:Downloading vectors from https://drive.google.com/uc?export=download&id=1Vih8gAmgBnuYDxfblbT94P6WjB7s1ZSh to /root/.vector_cache/wiki.en.bin
wiki.en.bin: 8.49GB [01:29, 94.8MB/s]
INFO:deepmatcher.data.field:Extracting vectors into /root/.vector_cache

Building vocabulary
0% [##############################] 100% | ETA: 00:00:00
Total time elapsed: 00:00:22

Computing principal components
0% [##############################] 100% | ETA: 00:00:00
Total time elapsed: 00:01:59


# Models

## DL

### Sif-Diff

In [22]:
# Create a sifDiff model.
model_sifdiff = dm.MatchingModel(attr_summarizer='sif')
model_sifdiff.initialize(train)  # Initilization

# Train ing model on 15 epochs, batch size of 16, positive-to-negative 
# ratio to be 1:1. We save the best model (with the 
# highest F1 score on the validation set) to 'sifDiff_model.pth'
startTime = time.time()
model_sifdiff.run_train(
    train,
    validation,
    epochs=10,
    batch_size=128,
    best_save_path='sifDiff_model.pth',
    pos_neg_ratio=2)
executionTime_sifDiff = (time.time() - startTime)/60

# Evaluate the accuracy on the test data.
model_sifdiff.run_eval(test)

/usr/local/lib/python3.7/dist-packages/torch/nn/modules/module.py:974: UserWarning: Using a non-full backward hook when the forward contains multiple autograd Nodes is deprecated and will be removed in future versions. This hook will be missing some grad_input. Please use register_full_backward_hook to get the documented behavior.
  warnings.warn("Using a non-full backward hook when the forward contains multiple autograd Nodes "


* Number of trainable parameters: 1083302
===>  TRAIN Epoch 1


/usr/local/lib/python3.7/dist-packages/torch/nn/functional.py:2742: UserWarning: reduction: 'mean' divides the total loss by both the batch size and the support size.'batchmean' divides only by the batch size, and aligns with the KL div math definition.'mean' will be changed to behave the same as 'batchmean' in the next major release.
  "reduction: 'mean' divides the total loss by both the batch size and the support size."
0% [██████████████████████████████] 100% | ETA: 00:00:00
Total time elapsed: 00:09:18


Finished Epoch 1 || Run Time:   52.4 | Load Time:  508.3 || F1:  97.38 | Prec:  95.20 | Rec:  99.67 || Ex/s: 222.13

===>  EVAL Epoch 1


0% [██████████████████████████████] 100% | ETA: 00:00:00
Total time elapsed: 00:02:56


...

0% [██████████████████████████████] 100% | ETA: 00:00:00
Total time elapsed: 00:02:51


Finished Epoch 10 || Run Time:   11.7 | Load Time:  159.8 || F1:  99.27 | Prec:  98.56 | Rec:  99.98 || Ex/s: 242.20

---------------------

Loading best model...
Training done.
===>  EVAL Epoch 4


0% [██████████████████████████████] 100% | ETA: 00:00:00
Total time elapsed: 00:03:03


Finished Epoch 4 || Run Time:   48.6 | Load Time:  135.0 || F1:  99.25 | Prec:  98.51 | Rec: 100.00 || Ex/s: 226.18



tensor(99.2504, device='cuda:0')

In [23]:
# Load the model.
model_sifdiff = dm.MatchingModel(attr_summarizer='sif')
model_sifdiff.load_state('sifDiff_model.pth')
# Load the candidate set. Note that the trained model is an input parameter as we need to trained 
# model for candidate set preprocessing.
predictions_sifDiff = model_sifdiff.run_prediction(test, output_attributes=list(test.get_raw_table().columns))

===>  PREDICT Epoch 4


0% [██████████████████████████████] 100% | ETA: 00:00:00
Total time elapsed: 00:03:01


Finished Epoch 4 || Run Time:   49.3 | Load Time:  132.9 || F1:  99.25 | Prec:  98.51 | Rec: 100.00 || Ex/s: 227.91



In [24]:
# get training data set and compute performance scores
train_n = em.read_csv_metadata(os.path.join('.', 'Dissertation_Data_2', 'train.csv'))
score_sifDiff = scores(executionTime_sifDiff, predictions_sifDiff, 0, train_n.shape[0], model = 'sif_diff')
prec_matrix = prec_matrix.append(score_sifDiff, ignore_index=True)

In [25]:
high_score_pairs = predictions_sifDiff[predictions_sifDiff['match_score'] >= 0.9].sort_values(by=['match_score'], ascending=False)

predictions_sifDiff['match_prediction'] = predictions_sifDiff['match_score'].apply(lambda score: 1 if score >= 0.5 else 0)
# Reorder columns to avoid scrolling...
predictions_sifDiff =  predictions_sifDiff[['match_score', 'match_prediction'] + predictions_sifDiff.columns.values[1:-1].tolist()]

In [26]:
path_predictions = os.path.join('.', 'Dissertation_Data_2', 'predictions_sifDiff.csv')
predictions_sifDiff.to_csv(path_predictions)

### RNN

In [27]:
# Create a sifDiff model.
model_rnn = dm.MatchingModel(attr_summarizer='rnn')
model_rnn.initialize(train)  # Initilization

# Train ing model on 15 epochs, batch size of 16, positive-to-negative 
# ratio to be 1:1. We save the best model (with the 
# highest F1 score on the validation set) to 'sifDiff_model.pth'.
startTime = time.time()
model_rnn.run_train(
    train,
    validation,
    epochs=10,
    batch_size=128,
    best_save_path='rnn_model.pth',
    pos_neg_ratio=2)
executionTime_rnn = (time.time() - startTime)/60

# Evaluate the accuracy on the test data.
model_rnn.run_eval(test)

* Number of trainable parameters: 7185302
===>  TRAIN Epoch 1


/usr/local/lib/python3.7/dist-packages/torch/nn/modules/module.py:949: UserWarning: Using non-full backward hooks on a Module that does not take as input a single Tensor or a tuple of Tensors is deprecated and will be removed in future versions. This hook will be missing some of the grad_input. Please use register_full_backward_hook to get the documented behavior.
  warnings.warn("Using non-full backward hooks on a Module that does not take as input a "
/usr/local/lib/python3.7/dist-packages/torch/nn/modules/module.py:974: UserWarning: Using a non-full backward hook when the forward contains multiple autograd Nodes is deprecated and will be removed in future versions. This hook will be missing some grad_input. Please use register_full_backward_hook to get the documented behavior.
  warnings.warn("Using a non-full backward hook when the forward contains multiple autograd Nodes "
/usr/local/lib/python3.7/dist-packages/torch/nn/modules/module.py:939: UserWarning: Using non-full backward h

Finished Epoch 1 || Run Time:  120.0 | Load Time:  557.2 || F1:  98.85 | Prec:  97.92 | Rec:  99.81 || Ex/s: 183.95

===>  EVAL Epoch 1


0% [██████████████████████████████] 100% | ETA: 00:00:00
Total time elapsed: 00:03:17


Finished Epoch 1 || Run Time:   18.7 | Load Time:  179.2 || F1:  99.25 | Prec:  98.52 | Rec:  99.99 || Ex/s: 209.76

* Best F1: tensor(99.2504, device='cuda:0')
Saving best model...
Done.
---------------------

===>  TRAIN Epoch 2


...

0% [██████████████████████████████] 100% | ETA: 00:00:00
Total time elapsed: 00:03:14


Finished Epoch 10 || Run Time:   18.2 | Load Time:  176.4 || F1:  99.32 | Prec:  98.74 | Rec:  99.90 || Ex/s: 213.33

* Best F1: tensor(99.3185, device='cuda:0')
Saving best model...
Done.
---------------------

Loading best model...
Training done.
===>  EVAL Epoch 10


0% [██████████████████████████████] 100% | ETA: 00:00:00
Total time elapsed: 00:03:31


Finished Epoch 10 || Run Time:   69.1 | Load Time:  143.3 || F1:  99.32 | Prec:  98.68 | Rec:  99.97 || Ex/s: 195.51



tensor(99.3170, device='cuda:0')

In [28]:
# Load the model.
model_rnn = dm.MatchingModel(attr_summarizer='rnn')
model_rnn.load_state('rnn_model.pth')
# Load the candidate set. Note that the trained model is an input parameter as we need to trained 
# model for candidate set preprocessing.
predictions_rnn = model_rnn.run_prediction(test, output_attributes=list(test.get_raw_table().columns))

===>  PREDICT Epoch 10


0% [██████████████████████████████] 100% | ETA: 00:00:00
Total time elapsed: 00:03:34


Finished Epoch 10 || Run Time:   71.3 | Load Time:  143.2 || F1:  99.32 | Prec:  98.68 | Rec:  99.97 || Ex/s: 193.58



In [ ]:
# get training data set and compute performance scores
score_rnn = scores(executionTime_rnn, predictions_rnn, 0, train_n.shape[0], model = 'rnn')
prec_matrix = prec_matrix.append(score_rnn, ignore_index=True)

In [30]:
high_score_pairs = predictions_rnn[predictions_rnn['match_score'] >= 0.9].sort_values(by=['match_score'], ascending=False)
predictions_rnn['match_prediction'] = predictions_rnn['match_score'].apply(lambda score: 1 if score >= 0.5 else 0)
# Reorder columns to avoid scrolling...
predictions_rnn =  predictions_rnn[['match_score', 'match_prediction'] + predictions_rnn.columns.values[1:-1].tolist()]

In [31]:
path_predictions = os.path.join('.', 'Dissertation_Data_2', 'predictions_rnn.csv')
predictions_rnn.to_csv(path_predictions)

### Attention

In [32]:
# Create a sifDiff model.
model_attention = dm.MatchingModel(attr_summarizer='attention')
model_attention.initialize(train)  # Initilization

# Train ing model on 15 epochs, batch size of 16, positive-to-negative 
# ratio to be 1:1. We save the best model (with the 
# highest F1 score on the validation set) to 'sifDiff_model.pth'.
startTime = time.time()
model_attention.run_train(
    train,
    validation,
    epochs=10,
    batch_size=128,
    best_save_path='attention_model.pth',
    pos_neg_ratio=2)
executionTime_attention = (time.time() - startTime)/60

# Evaluate the accuracy on the test data.
model_attention.run_eval(test)

/usr/local/lib/python3.7/dist-packages/torch/nn/modules/module.py:974: UserWarning: Using a non-full backward hook when the forward contains multiple autograd Nodes is deprecated and will be removed in future versions. This hook will be missing some grad_input. Please use register_full_backward_hook to get the documented behavior.
  warnings.warn("Using a non-full backward hook when the forward contains multiple autograd Nodes "
/usr/local/lib/python3.7/dist-packages/torch/nn/modules/module.py:949: UserWarning: Using non-full backward hooks on a Module that does not take as input a single Tensor or a tuple of Tensors is deprecated and will be removed in future versions. This hook will be missing some of the grad_input. Please use register_full_backward_hook to get the documented behavior.
  warnings.warn("Using non-full backward hooks on a Module that does not take as input a "


* Number of trainable parameters: 15069302
===>  TRAIN Epoch 1


/usr/local/lib/python3.7/dist-packages/torch/nn/modules/module.py:939: UserWarning: Using non-full backward hooks on a Module that does not return a single Tensor or a tuple of Tensors is deprecated and will be removed in future versions. This hook will be missing some of the grad_output. Please use register_full_backward_hook to get the documented behavior.
  warnings.warn("Using non-full backward hooks on a Module that does not return a "
/usr/local/lib/python3.7/dist-packages/torch/nn/functional.py:2742: UserWarning: reduction: 'mean' divides the total loss by both the batch size and the support size.'batchmean' divides only by the batch size, and aligns with the KL div math definition.'mean' will be changed to behave the same as 'batchmean' in the next major release.
  "reduction: 'mean' divides the total loss by both the batch size and the support size."
0% [██████████████████████████████] 100% | ETA: 00:00:00
Total time elapsed: 00:13:15


Finished Epoch 1 || Run Time:  253.0 | Load Time:  545.6 || F1:  98.05 | Prec:  96.49 | Rec:  99.66 || Ex/s: 155.97

===>  EVAL Epoch 1


0% [██████████████████████████████] 100% | ETA: 00:00:00
Total time elapsed: 00:03:51


...

0% [██████████████████████████████] 100% | ETA: 00:00:00
Total time elapsed: 00:03:46


Finished Epoch 10 || Run Time:   48.3 | Load Time:  178.6 || F1:  99.31 | Prec:  98.77 | Rec:  99.85 || Ex/s: 183.01

* Best F1: tensor(99.3067, device='cuda:0')
Saving best model...
Done.
---------------------

Loading best model...
Training done.
===>  EVAL Epoch 10


0% [██████████████████████████████] 100% | ETA: 00:00:00
Total time elapsed: 00:05:32


Finished Epoch 10 || Run Time:  188.4 | Load Time:  145.1 || F1:  99.28 | Prec:  98.70 | Rec:  99.86 || Ex/s: 124.50



tensor(99.2761, device='cuda:0')

In [33]:
# Load the model.
model_attention = dm.MatchingModel(attr_summarizer='attention')
model_attention.load_state('attention_model.pth')
# Load the candidate set. Note that the trained model is an input parameter as we need to trained 
# model for candidate set preprocessing.
predictions_attention = model_attention.run_prediction(test, output_attributes=list(test.get_raw_table().columns))

===>  PREDICT Epoch 10


0% [██████████████████████████████] 100% | ETA: 00:00:00
Total time elapsed: 00:05:31


Finished Epoch 10 || Run Time:  188.4 | Load Time:  144.1 || F1:  99.28 | Prec:  98.70 | Rec:  99.86 || Ex/s: 124.90



In [34]:
# get training data set and compute performance scores
score_attention = scores(executionTime_attention, predictions_attention, 0, train_n.shape[0], model = 'attention')
prec_matrix = prec_matrix.append(score_attention, ignore_index=True)

In [35]:
high_score_pairs = predictions_attention[predictions_attention['match_score'] >= 0.9].sort_values(by=['match_score'], ascending=False)
predictions_attention['match_prediction'] = predictions_attention['match_score'].apply(lambda score: 1 if score >= 0.5 else 0)
# Reorder columns to avoid scrolling...
predictions_attention =  predictions_attention[['match_score', 'match_prediction'] + predictions_attention.columns.values[1:-1].tolist()]

In [36]:
path_predictions = os.path.join('.', 'Dissertation_Data_2', 'predictions_att.csv')
predictions_attention.to_csv(path_predictions)

### Hybrid

In [37]:
# Create a sifDiff model.
model_hyb = dm.MatchingModel(attr_summarizer='hybrid')
model_hyb.initialize(train)  # Initilization

# Train ing model on 15 epochs, batch size of 16, positive-to-negative 
# ratio to be 1:1. We save the best model (with the 
# highest F1 score on the validation set) to 'sifDiff_model.pth'.
startTime = time.time()
model_hyb.run_train(
    train,
    validation,
    epochs=10,
    batch_size=128,
    best_save_path='hybrid_model.pth',
    pos_neg_ratio=2)
executionTime_hyb = (time.time() - startTime)/60

# Evaluate the accuracy on the test data.
model_hyb.run_eval(test)

/usr/local/lib/python3.7/dist-packages/torch/nn/modules/module.py:949: UserWarning: Using non-full backward hooks on a Module that does not take as input a single Tensor or a tuple of Tensors is deprecated and will be removed in future versions. This hook will be missing some of the grad_input. Please use register_full_backward_hook to get the documented behavior.
  warnings.warn("Using non-full backward hooks on a Module that does not take as input a "
/usr/local/lib/python3.7/dist-packages/torch/nn/modules/module.py:974: UserWarning: Using a non-full backward hook when the forward contains multiple autograd Nodes is deprecated and will be removed in future versions. This hook will be missing some grad_input. Please use register_full_backward_hook to get the documented behavior.
  warnings.warn("Using a non-full backward hook when the forward contains multiple autograd Nodes "


* Number of trainable parameters: 33136817
===>  TRAIN Epoch 1


/usr/local/lib/python3.7/dist-packages/torch/nn/modules/module.py:939: UserWarning: Using non-full backward hooks on a Module that does not return a single Tensor or a tuple of Tensors is deprecated and will be removed in future versions. This hook will be missing some of the grad_output. Please use register_full_backward_hook to get the documented behavior.
  warnings.warn("Using non-full backward hooks on a Module that does not return a "
/usr/local/lib/python3.7/dist-packages/torch/nn/functional.py:2742: UserWarning: reduction: 'mean' divides the total loss by both the batch size and the support size.'batchmean' divides only by the batch size, and aligns with the KL div math definition.'mean' will be changed to behave the same as 'batchmean' in the next major release.
  "reduction: 'mean' divides the total loss by both the batch size and the support size."
0% [██████████████████████████████] 100% | ETA: 00:00:00
Total time elapsed: 00:16:34


Finished Epoch 1 || Run Time:  454.0 | Load Time:  544.6 || F1:  98.51 | Prec:  97.20 | Rec:  99.85 || Ex/s: 124.74

===>  EVAL Epoch 1


0% [██████████████████████████████] 100% | ETA: 00:00:00
Total time elapsed: 00:04:15


...

0% [██████████████████████████████] 100% | ETA: 00:00:00
Total time elapsed: 00:04:07


Finished Epoch 10 || Run Time:   73.5 | Load Time:  174.2 || F1:  99.31 | Prec:  98.82 | Rec:  99.82 || Ex/s: 167.68

---------------------

Loading best model...
Training done.
===>  EVAL Epoch 8


0% [██████████████████████████████] 100% | ETA: 00:00:00
Total time elapsed: 00:07:03


Finished Epoch 8 || Run Time:  282.5 | Load Time:  141.8 || F1:  99.27 | Prec:  98.67 | Rec:  99.89 || Ex/s:  97.86



tensor(99.2744, device='cuda:0')

In [38]:
# Load the model.
model_hyb = dm.MatchingModel(attr_summarizer='hybrid')
model_hyb.load_state('hybrid_model.pth')
# Load the candidate set. Note that the trained model is an input parameter as we need to trained 
# model for candidate set preprocessing.
predictions_hyb = model_hyb.run_prediction(test, output_attributes=list(test.get_raw_table().columns))

===>  PREDICT Epoch 8


0% [██████████████████████████████] 100% | ETA: 00:00:00
Total time elapsed: 00:07:05


Finished Epoch 8 || Run Time:  285.5 | Load Time:  141.4 || F1:  99.27 | Prec:  98.67 | Rec:  99.89 || Ex/s:  97.27



In [39]:
# get training data set and compute performance scores
score_hyb = scores(executionTime_hyb, predictions_hyb, 0, train_n.shape[0], model = 'hybrid')
prec_matrix =prec_matrix.append(score_hyb, ignore_index=True)

In [40]:
high_score_pairs = predictions_hyb[predictions_hyb['match_score'] >= 0.9].sort_values(by=['match_score'], ascending=False)
predictions_hyb['match_prediction'] = predictions_hyb['match_score'].apply(lambda score: 1 if score >= 0.5 else 0)
# Reorder columns to avoid scrolling...
predictions_hyb =  predictions_hyb[['match_score', 'match_prediction'] + predictions_hyb.columns.values[1:-1].tolist()]

In [41]:
path_DL_res = os.path.join('.', 'Dissertation_Data_2', 'DL_res.csv')
prec_matrix.to_csv(path_DL_res)

In [42]:
path_predictions = os.path.join('.', 'Dissertation_Data_2', 'predictions_hyb.csv')
predictions_hyb.to_csv(path_predictions)

## ML

In [43]:
Cora.to_csv(os.path.join('.', 'Dissertation_Data_2', 'cora.csv'),index=False)
Cora = em.read_csv_metadata(os.path.join('.', 'Dissertation_Data_2', 'cora.csv'), key='id')

In [44]:
path_cand = os.path.join('.', 'Dissertation_Data_2', 'cand.csv')
cand.to_csv(path_cand,index=False)

In [45]:
# creating dataset for ML matching
S = em.read_csv_metadata(path_cand, 
                         key='_id',
                         ltable=Cora, rtable=Cora, 
                         fk_ltable='ltable_id', fk_rtable='rtable_id')

# Split S into training and test data
IJ = em.split_train_test(S, train_proportion=0.5, random_state=0)
I = IJ['train']
J = IJ['test']

# Generate a set of features
F = em.get_features_for_matching(Cora, Cora, validate_inferred_attr_types=False)

# Convert the I into a set of feature vectors using F
H = em.extract_feature_vecs(I, 
                            feature_table=F, 
                            attrs_after='label',
                            show_progress=False)

# Create a set of ML-matchers
dt = em.DTMatcher(name='DecisionTree', random_state=0)
svm = em.SVMMatcher(name='SVM', random_state=0)
rf = em.RFMatcher(name='RF', random_state=0)
lg = em.LogRegMatcher(name='LogReg', random_state=0)
ln = em.LinRegMatcher(name='LinReg')

# Impute feature vectors with the mean of the column values.
H = em.impute_table(H, exclude_attrs=['_id', 'ltable_id', 'rtable_id', 'label'], missing_val = np.nan, strategy='mean')

/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:3361: DtypeWarning: Columns (7,9,13,15,17,18,20,21,22,23,24,25,27,28,30,31,32) have mixed types.Specify dtype option on import or set low_memory=False.
  if (await self.run_code(code, result,  async_=asy)):
/usr/local/lib/python3.7/dist-packages/py_entitymatching/matcher/matcherutils.py:224: FutureWarning: The pandas.np module is deprecated and will be removed from pandas in a future version. Import numpy directly instead
  imp.statistics_[pd.np.isnan(imp.statistics_)] = val_all_nans


In [46]:
# Select the best ML matcher using CV
result = em.select_matcher([dt, rf, svm, ln, lg], table=H, 
        exclude_attrs=['_id', 'ltable_id', 'rtable_id', 'label'],
        k=10,
        target_attr='label', metric_to_select_matcher='f1', random_state=0)
result['cv_stats']

/usr/local/lib/python3.7/dist-packages/joblib/externals/loky/process_executor.py:691: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  "timeout or by a memory leak.", UserWarning


,Matcher,Average precision,Average recall,Average f1
0,DecisionTree,0.998789,0.998665,0.998727
1,RF,0.999425,0.999301,0.999363
2,SVM,0.920802,0.973276,0.946311
3,LinReg,0.916783,0.977912,0.946357
4,LogReg,0.941104,0.957282,0.949109


In [47]:
result['drill_down_cv_stats']['precision']

,Name,Matcher,Num folds,Fold 1,Fold 2,Fold 3,Fold 4,Fold 5,Fold 6,Fold 7,Fold 8,Fold 9,Fold 10,Mean score
0,DecisionTree,<py_entitymatching.matcher.dtmatcher.DTMatcher object at 0x7feb89be5bd0>,10,0.998445,0.998915,0.998275,0.998744,0.998592,0.999370,0.998160,0.998923,0.999236,0.999225,0.998789
1,RF,<py_entitymatching.matcher.rfmatcher.RFMatcher object at 0x7feb89be5fd0>,10,0.999223,0.999380,0.999216,0.999529,0.999687,0.999371,0.999233,0.999538,0.999542,0.999535,0.999425
2,SVM,<py_entitymatching.matcher.svmmatcher.SVMMatcher object at 0x7fed0cb613d0>,10,0.923767,0.918326,0.919395,0.920059,0.918663,0.919345,0.923750,0.922360,0.923277,0.919073,0.920802
3,LinReg,<py_entitymatching.matcher.linregmatcher.LinRegMatcher object at 0x7fed01ead810>,10,0.922139,0.917235,0.914819,0.916728,0.914052,0.914353,0.916691,0.917208,0.921182,0.913421,0.916783
4,LogReg,<py_entitymatching.matcher.logregmatcher.LogRegMatcher object at 0x7feb89be5e10>,10,0.954116,0.937386,0.944051,0.939833,0.947106,0.927430,0.950272,0.942197,0.934694,0.933951,0.941104


In [ ]:
path_ML_res = os.path.join('.', 'Dissertation_Data_2', 'ML_res.csv')

result['cv_stats'].to_csv(path_ML_res)

In [1]:
i = 0
try:
  while True:
    i += 1
  if not i % 1000:
    gc.collect()
except KeyboardInterrupt:
 print(i)

138031597
